# Triggering with a threshold on channel
For the first example in the acquisition section, we will take a look at the most common way of capturing data - acquiring data when the signal reaches a specified voltage threshold.

**Note:**  
The voltage range of fast analog inputs on the Red Pitaya depends on the input jumper position. HV sets the input range to ±20 V, while LV sets the input range to ±1 V. For more information, please read the following [chapter](https://redpitaya.readthedocs.io/en/latest/developerGuide/hardware/125-14/fastIO.html#analog-inputs).

Create a loop-back from fast analog outputs to fast analog inputs, as shown in the picture below.  
Please make sure the jumpers are set to ±1 V (LV).

![Fast loop back](../img/FastIOLoopBack.png "Example of the fast loop back.")

## Libraries and FPGA image

We need the additional functionality of *numpy* and *matplotlib* for data plotting and faster array operations.

In [ ]:
import time
import numpy as np
from matplotlib import pyplot as plt
from rp_overlay import overlay
import rp

fpga = overlay()
rp.rp_Init()

## Macros
Throughout this tutorial we will mention macros multiple times. Here is a complete list of macros that will come in handy when customising this notebook. The marcos are a part of the **rp** library.

- **Decimation** - RP_DEC_1, RP_DEC_2, RP_DEC_4, RP_DEC_8, RP_DEC_16, RP_DEC_32, RP_DEC_64, RP_DEC_128, RP_DEC_256, RP_DEC_512, RP_DEC_1024, RP_DEC_2048, RP_DEC_4096, RP_DEC_8192, RP_DEC_16384, RP_DEC_32768, RP_DEC_65536 
- **Acquisition trigger** - RP_TRIG_SRC_DISABLED, RP_TRIG_SRC_NOW, RP_TRIG_SRC_CHA_PE, RP_TRIG_SRC_CHA_NE, RP_TRIG_SRC_CHB_PE, RP_TRIG_SRC_CHB_NE, RP_TRIG_SRC_EXT_PE, RP_TRIG_SRC_EXT_NE, RP_TRIG_SRC_AWG_PE, RP_TRIG_SRC_AWG_NE
- **Acquisition trigger state** - RP_TRIG_STATE_TRIGGERED, RP_TRIG_STATE_WAITING
- **Buffer size** - ADC_BUFFER_SIZE, DAC_BUFFER_SIZE
- **Fast analog channels** - RP_CH_1, RP_CH_2

SIGNALlab 250-12 only:
- **Input coupling** - RP_DC, RP_AC

STEMlab 125-14 4-Input only:
- **Fast analog channels** - RP_CH_3, RP_CH_4
- **Acquisition trigger** - RP_TRIG_SRC_CHC_PE, RP_TRIG_SRC_CHC_NE, RP_TRIG_SRC_CHD_PE, RP_TRIG_SRC_CHD_NE


## Theory
When it comes to data acquisition, there are certain terms we must be familiar with to understand how the acquisition works and correctly set it up. Here are a few terms that might be unknown to beginners in this field:

- **Triggering moment** - The point in time where the trigger condition is met and the instrument starts capturing data. 
- **Trigger condition** - Composed of trigger level and signal front.
- **Trigger level** - the value of the input signal in Volts at which the acquisition will start
- **Signal front** - also reffered to as **positive/negative edge** (mostly used for digital logic). The point in time where a signal becomes *high* is called a positive edge, while the moment it becomes *low* is a negative edge. With analog signals, this refers to how the signal crosses the trigger level. A positive edge means the signal must be lower and rise to the trigger level, and for a negative edge, the signal must be higher and fall to the trigger level to meet the trigger condition.  
- **Decimation** - Refers to how many samples are skipped before a sample is saved. For example, if we set decimation to 10, Red Pitaya saves every tenth sample, while the other nine are discarded. Indirectly defines the sampling frequency.
- **Averaging** - When decimation is higher than one, each saved sample is an average of all skipped samples. By default, this is turned *on*.
- **Acquisition units** - The units can either be set to *RAW*, where the value is taken directly from the Red Pitaya's ADC, or *Volts*, where Red Pitaya automatically performs the conversion. By default, this is set to *Volts*.
- **Trigger delay** - Defines how many samples the triggering moment is delayed. If the trigger delay is set to 0, the triggering moment is in the middle of the returned data buffer. A positive trigger delay moves the triggering moment towards the start of the buffer, and a negative trigger delay moves it towards the end of the buffer. By default, this is set to 0.

![Python buffer](../img/python_buffer.png "Acquisition buffer")

## Trigger level

For the purposes of demonstartion we will first generate a signal on Red Pitaya's outputs and then acquire it on the inputs. The so-called "loopback" does not have many applications in practice, but is perfect for this demonstration. In practice, an external signal is applied directly to the Red Pitaya's inputs.

In [ ]:
# Generator parameters
channel = rp.RP_CH_1
channel2 = rp.RP_CH_2
waveform = rp.RP_WAVEFORM_SINE
freq = 100000
ampl = 1.0

# Acquisition paramters
dec = rp.RP_DEC_1

trig_lvl = 0.5
trig_dly = 0

acq_trig_sour = rp.RP_TRIG_SRC_CHA_PE
N = 16384

Above are the parameters for the generation and acquisition. We set the acquisition trigger to IN1 positive edge 0.5 V.

Now, we are going to reset both acquisition and generation parameters and configure the generator.

In [ ]:
rp.rp_GenReset()
rp.rp_AcqReset()

###### Generation #####
# OUT1
print("Gen_start")
rp.rp_GenWaveform(channel, waveform)
rp.rp_GenFreqDirect(channel, freq)
rp.rp_GenAmp(channel, ampl)

# OUT2
rp.rp_GenWaveform(channel2, waveform)
rp.rp_GenFreqDirect(channel2, freq)
rp.rp_GenAmp(channel2, ampl)

# Specify generator trigger source
rp.rp_GenTriggerSource(channel, rp.RP_GEN_TRIG_SRC_INTERNAL)

# Enable output synchronisation
rp.rp_GenOutEnableSync(True)

Here comes the new part - Configuring the acquisition parameters.

In [ ]:

##### Acquisition #####
# Set Decimation
rp.rp_AcqSetDecimation(rp.RP_DEC_1)

# Set trigger level and delay
rp.rp_AcqSetTriggerLevel(rp.RP_T_CH_1, trig_lvl)
rp.rp_AcqSetTriggerDelay(trig_dly)

After configuring the acquisition parameters, we need to start the acquisition and set the acquisition trigger source. The order of the two commands is extremely important here! **Acquisition must be started before the acquisition trigger is specified!**

In [ ]:
# Start Acquisition
print("Acq_start")
rp.rp_AcqStart()
time.sleep(0.1)
# Specify trigger - input 1 positive edge
rp.rp_AcqSetTriggerSrc(acq_trig_sour)
time.sleep(0.1)

Now, we will trigger the acquisition (by triggering the generator), and acquire the data.


In [ ]:
rp.rp_GenTriggerOnly(channel)       # Trigger generator

# Trigger state
while 1:
    trig_state = rp.rp_AcqGetTriggerState()[1]
    if trig_state == rp.RP_TRIG_STATE_TRIGGERED:
        break

# Fill state
while 1:
    if rp.rp_AcqGetBufferFillState()[1]:
        break


# Get data
# RAW
ibuff = rp.i16Buffer(N)
res = rp.rp_AcqGetOldestDataRaw(rp.RP_CH_1, N, ibuff.cast())[1]

# Volts
fbuff = rp.fBuffer(N)
res = rp.rp_AcqGetOldestDataV(rp.RP_CH_1, N, fbuff)[1]

With *rp_AcqGetTriggerState()* and *rp_AcqGetBufferFillState()*, we are checking the second returned parameter (the first one is function execution success).

After confirming that the trigger condition was met and that the data buffer is full, the data can be acquired from the Red Pitaya. We can use either of the two presented functions, depending on whether we want RAW data or Volts directly.

Next, we will convert the data to a numpy array for easy plotting.

In [ ]:
data_V = np.zeros(N, dtype = float)
data_raw = np.zeros(N, dtype = int)
X = np.arange(0, N, 1)

for i in range(0, N, 1):
    data_V[i] = fbuff[i]
    data_raw[i] = ibuff[i]

figure, axis = plt.subplots(1, 2) 

axis[0].plot(X, data_V) 
axis[0].set_title("Volts")

axis[1].plot(X, data_raw) 
axis[1].set_title("RAW") 

plt.show()

In [ ]:
# Release resources
rp.rp_Release()

And here is the data we have been looking for, nicely displayed in a graph. As you can see from the X-axis, Red Pitaya returns a 16384-sample long buffer of acquired data. The triggering moment is in the middle, around the 8192th sample (the trigger delay is 0), so half the data was acquired before the trigger and half the data after it. If the transition events are not what we are looking for, we can increase the trigger delay to get more data points after the triggering moment or decrease it (negative numbers) to get more information before the triggering event.

### Note
There are a lot of different commands for the Acquisition. The list of available functions is quite an achievement to read through, so from now on, please refer to the *C and Python API section* of the [SCPI & SPI command list](https://redpitaya.readthedocs.io/en/latest/appsFeatures/remoteControl/command_list.html#list-of-supported-scpi-api-commands) for all available commands.
